# Introducción, objetivos y contenido

Este trabajo corresponde a la fase de ETL (Extraction, Transformation and Loading). El objetivo de esta fase es obtener datasets limpios y listos para ser utilizado en fases posteriores del proyecto. 

Contenidos:
* Importación de librerías
* Carga de datos
* Preparación de datos para cada dataset 
    * Ingeniería de características
    * Verificación de tipos de datos
    * Valores duplicados
    * Valores nulos
* Exportación de los datasets limpios
* Armado y exportación de dataframes para API

# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
from math import factorial
from scipy import stats as st
import json
import gzip
import ast
from pandas import json_normalize
from textblob import TextBlob
import re

# Carga de datos

Tenemos un total de 3 datasets en formato json comprimido, debido a ello haremos la carga de datos en forma separada para tomar los recaudos correspondientes.

Dataset GAMES: Este archivo ha sido posible cargarlo en formato jason descomprimido, por lo cual su código resulta simple.

In [2]:
steam_games = pd.read_json('steam_games.json', lines=True)

Dataset REVIEWS: Dado que este dataset tiene una estructura menos estandarizada, ha sido necesario cargarlo aplicando un código que estandarice cada línea del archivo. Luego esas líneas se incorporan como lista a una variable archivo que recopila los datos originales transformados.

In [3]:
dataset_list_reviews = []
with gzip.open('user_reviews.json.gz', 'rb') as file:
    for line in file:
        dataset_list_reviews.append(ast.literal_eval(line.decode('utf-8')))
user_reviews = pd.DataFrame(dataset_list_reviews)
file.close()

Dataset ITEMS: Dado que este dataset tiene una estructura menos estandarizada, ha sido necesario cargarlo aplicando un código que estandarice cada línea del archivo. Luego esas líneas se incorporan como lista a una variable archivo que recopila los datos originales transformados.

In [4]:
dataset_list_items = []
with gzip.open('users_items.json.gz', 'rb') as file:
    for line in file:
        dataset_list_items.append(ast.literal_eval(line.decode('utf-8')))
user_items = pd.DataFrame(dataset_list_items)
file.close()

# Preparación de datos

## Dataset GAMES

### Ingeniería de características - Dataset GAMES

In [5]:
df_games = steam_games
df_games.sample(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
95513,Sometimes You,"[Casual, Indie]",Black Sand Drift Collector's Edition Content,Black Sand Drift Collector's Edition Content,http://store.steampowered.com/app/517560/Black...,2016-09-08,"[Indie, Casual]",http://steamcommunity.com/app/517560/reviews/?...,"[Single-player, Downloadable Content, Steam Tr...",1.99,0.0,517560.0,Echo Hall Studios
7944,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [6]:
# Renombramiento del campo "id"
df_games.rename(columns={'id': 'item_id'}, inplace=True)

In [7]:
# Desagregación de campos cuyos valores son listas
df_games = df_games.explode('genres')
df_games = df_games.explode('tags')
df_games = df_games.explode('specs')
df_games.sample(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
115194,None,Adventure,Hero Siege - The Depths of Hell (Collector's E...,Hero Siege - The Depths of Hell (Collector's E...,http://store.steampowered.com/app/331181/Hero_...,2015-04-30,RPG,http://steamcommunity.com/app/331181/reviews/?...,Steam Achievements,4.99,0.0,331181.0,"Elias Viglione,Jussi Kukkonen"
92281,Pixel Ferrets,Indie,Secrets of Grindea Demo,Secrets of Grindea Demo,http://store.steampowered.com/app/372500/Secre...,2015-07-13,Adventure,http://steamcommunity.com/app/372500/reviews/?...,Multi-player,None,0.0,372500.0,Pixel Ferrets
96785,None,Casual,Rocksmith® 2014 Edition – Remastered – blink-1...,Rocksmith® 2014 Edition – Remastered – blink-1...,http://store.steampowered.com/app/509703/Rocks...,2017-01-17,Casual,http://steamcommunity.com/app/509703/reviews/?...,Steam Cloud,2.99,0.0,509703.0,Ubisoft - San Francisco
108564,argagonky,Casual,Space Cat,Space Cat,http://store.steampowered.com/app/539980/Space...,2016-12-02,Fast-Paced,http://steamcommunity.com/app/539980/reviews/?...,Multi-player,4.99,0.0,539980.0,argagonky
101194,Ripstone,Simulation,Chess Ultra Easter Island chess set,Chess Ultra Easter Island chess set,http://store.steampowered.com/app/639000/Chess...,2017-12-20,Strategy,http://steamcommunity.com/app/639000/reviews/?...,Full controller support,Free,0.0,639000.0,Ripstone


In [8]:
# Agregación del campo "year"
default_date = pd.to_datetime('1900-01-01')  # Imputar un valor predeterminado en lugar de los valores no válidos en 'release_date'
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce').fillna(default_date)

df_games['release_date'] = pd.to_datetime(df_games['release_date'])     # Convertir la columna 'release_date' a objetos de fecha y hora
df_games['year'] = df_games['release_date'].dt.year
df_games = df_games[df_games['year'] != 1900]
df_games.sample(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year
111744,Phung Games,Action,Tank Brawl,Tank Brawl,http://store.steampowered.com/app/436870/Tank_...,2016-04-20,Action,http://steamcommunity.com/app/436870/reviews/?...,Co-op,4.99,0.0,436870.0,Phung Games,2016
119424,None,Casual,Rochard,Rochard,http://store.steampowered.com/app/107800/Rochard/,2011-11-15,Space,http://steamcommunity.com/app/107800/reviews/?...,Steam Cloud,9.99,0.0,107800.0,Recoil Games,2011
91176,Deep Silver,Adventure,Escape Dead Island,Escape Dead Island,http://store.steampowered.com/app/226560/Escap...,2014-11-18,Horror,http://steamcommunity.com/app/226560/reviews/?...,Steam Trading Cards,14.99,0.0,226560.0,Fatshark,2014
115754,Retrific,Indie,Just Get Through,Just Get Through,http://store.steampowered.com/app/338190/Just_...,2015-01-30,Level Editor,http://steamcommunity.com/app/338190/reviews/?...,Captions available,4.99,0.0,338190.0,Retrific,2015
100079,CCP,Strategy,EVE Online: 6 Months Omega Time,EVE Online: 6 Months Omega Time,http://store.steampowered.com/app/695742/EVE_O...,2017-10-03,Massively Multiplayer,http://steamcommunity.com/app/695742/reviews/?...,Downloadable Content,71.7,0.0,695742.0,CCP,2017


In [9]:
# Eliminación de campos que no serán utilizados
#df_games_eliminarcampos = ['url', 'title','release_date', 'reviews_url', 'specs', ]
# df_games = df_games.drop(df_games_eliminarcampos, axis=1)

In [10]:
# Filtrado de campos a utilizar
df_games = df_games[['item_id', 'app_name', 'genres', 'year', 'price', 'developer']]
df_games.sample(5)

,item_id,app_name,genres,year,price,developer
99416,687160.0,White Day - Horror Costume - Ji-Min Yoo,Adventure,2017,1.99,SONNORI Corp
93298,423852.0,Depth - Soundtrack,Action,2015,4.99,Digital Confectioners
116383,332020.0,Fantasy Grounds - C&C: A5 The Shattered Horn,Strategy,2014,3.99,"SmiteWorks USA, LLC"
108715,302670.0,Call to Arms,Indie,2016,Free to Play,Digitalmindsoft
102414,560300.0,Thy Sword,Adventure,2017,8.99,GamePhase


### Verificación de tipos de datos - Dataset GAMES

In [11]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1983295 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   item_id    float64
 1   app_name   object 
 2   genres     object 
 3   year       int32  
 4   price      object 
 5   developer  object 
dtypes: float64(1), int32(1), object(4)
memory usage: 98.4+ MB


In [12]:
# Conversión de tipos de datos

df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')  # Conversión a tipo numérico, forzando los errores a NaN
#df_games['early_access'] = df_games['early_access'].astype(bool)       # Conversión a tipo booleano
df_games['item_id'] = pd.to_numeric(df_games['item_id'], errors='coerce')        # Conversión a tipo entero, forzando los errores a NaN
df_games['item_id'].fillna(0, inplace=True)
df_games['item_id'] = df_games['item_id'].astype(int)
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1983295 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   item_id    int64  
 1   app_name   object 
 2   genres     object 
 3   year       int32  
 4   price      float64
 5   developer  object 
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 98.4+ MB


In [13]:
df_games.sample(5)

,item_id,app_name,genres,year,price,developer
101367,408410,X-Morph: Defense,Strategy,2017,19.99,EXOR Studios
115166,355050,Sky Force Anniversary,Indie,2015,9.99,Infinite Dreams
110937,489330,The Lady - Wallpaper Pack,Indie,2016,NaN,"MPR ART Hallucinations,Roger Levy"
117442,236430,DARK SOULS™ II,RPG,2014,39.99,"FromSoftware, Inc"
115819,319000,Hyperdimension Neptunia Re;Birth1 Histy's Tria...,Strategy,2015,NaN,"IDEA FACTORY Co., Ltd.,COMPILE HEART Co., Ltd...."


### Verficación de valores duplicados - Dataset GAMES

In [14]:
df_games.duplicated().sum()

1911358

In [15]:
df_games = df_games.drop_duplicates().reset_index(drop=True)
df_games = df_games.drop_duplicates(subset=['item_id', 'app_name'])     # Eliminar filas cuyo campo "item_id" y "app_name" tiene duplicados
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29782 entries, 0 to 71935
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_id    29782 non-null  int64  
 1   app_name   29781 non-null  object 
 2   genres     28548 non-null  object 
 3   year       29782 non-null  int32  
 4   price      27210 non-null  float64
 5   developer  28532 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 1.5+ MB


### Verificación de valores nulos - Dataset GAMES

In [16]:
df_games.isnull().sum()

item_id         0
app_name        1
genres       1234
year            0
price        2572
developer    1250
dtype: int64

En este marco es difícil establecer un patrón de valores nulos que nos permita comprender la razón de su existencia. Considerando los efectos para los que es necesario este dataset, optaremos por eliminar los registros cuyos campos prinicipales del dataset presenten valores nulos.

In [17]:
df_games = df_games.dropna(subset=['genres', 'app_name', 'price', 'developer'])

In [18]:
df_games.isnull().sum()

item_id      0
app_name     0
genres       0
year         0
price        0
developer    0
dtype: int64

## Dataset REVIEWS

### Ingeniería de características - Dataset REVIEWS

In [19]:
df_reviews = user_reviews
df_reviews.sample(2)

,user_id,user_url,reviews
8790,76561198072074537,http://steamcommunity.com/profiles/76561198072...,"[{'funny': '', 'posted': 'Posted February 25, ..."
8437,Zychratic,http://steamcommunity.com/id/Zychratic,"[{'funny': '', 'posted': 'Posted August 1, 201..."


In [20]:
# Desagregación del campo "reviews"

df_reviews = user_reviews.explode('reviews')
df_reviews = pd.concat([df_reviews.drop(['reviews'], axis=1), df_reviews['reviews'].apply(pd.Series)], axis=1)
df_reviews.sample(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
17744,76561198177315429,http://steamcommunity.com/profiles/76561198177...,,"Posted December 8, 2015.",,730,0 of 2 people (0%) found this review helpful,True,Best Game I've Ever Played! MUST PLAY! R8 IT 8...,NaN
23509,76561198083126313,http://steamcommunity.com/profiles/76561198083...,,"Posted November 20, 2014.",,238460,No ratings yet,True,FUNNY AWESOME AND GREAT PUZZELplay this with y...,NaN
2784,nathan3197,http://steamcommunity.com/id/nathan3197,,"Posted December 7, 2013.",,620,1 of 1 people (100%) found this review helpful,True,fun game and you can make your own map and dow...,NaN
20256,76561198049458838,http://steamcommunity.com/profiles/76561198049...,,"Posted June 26, 2014.",,620,No ratings yet,True,"this was a really fun game, i absolutely enjoy...",NaN
12376,76561198075915471,http://steamcommunity.com/profiles/76561198075...,,"Posted August 16, 2015.",,8930,No ratings yet,True,"PLEASE IT IS SUCH A GOOD FRICKING GAME, HOLY C...",NaN


In [21]:
# Agregación de los campos "date" y "year"

def extract_posted_date(posted_str):         # Función para extraer la fecha del campo "posted"
    pattern = r'Posted (\w+ \d{1,2}, \d{4})' # Definición del patrón observado
    match = re.search(pattern, posted_str)
    if match:
        return match.group(1)
    else:
        return None

# Aplicar la función para extraer la fecha del campo "posted"
df_reviews['posted_date'] = df_reviews['posted'].apply(lambda x: np.nan if pd.isna(x) else extract_posted_date(x))

df_reviews['posted_date'] = pd.to_datetime(df_reviews['posted_date'])
df_reviews['year_review'] = df_reviews['posted_date'].dt.year
df_reviews['year_review'] = df_reviews['year_review'].fillna(0)
df_reviews['year_review'] = df_reviews['year_review'].astype(int)

df_reviews.sample(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0,posted_date,year_review
17037,76561198026229747,http://steamcommunity.com/profiles/76561198026...,,Posted February 5.,,310380,0 of 1 people (0%) found this review helpful,False,"it seems like a good game and all, but if your...",NaN,NaT,0
23168,76561198079642013,http://steamcommunity.com/profiles/76561198079...,,"Posted December 28, 2012.",,440,1 of 1 people (100%) found this review helpful,True,DOWNLOAD IT,NaN,2012-12-28,2012


In [22]:
# Análisis de sentimientos a partir del campo "review"

df_reviews['review'] = df_reviews['review'].astype(str)
df_reviews['polarity'] = df_reviews['review'].apply(lambda text: TextBlob(text).sentiment.polarity)
df_reviews['sentiment'] = pd.cut(df_reviews['polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[0, 1, 2])

In [23]:
df_reviews.sample(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0,posted_date,year_review,polarity,sentiment
21369,alifyandra,http://steamcommunity.com/id/alifyandra,,"Posted March 14, 2014.",,211820,1 of 1 people (100%) found this review helpful,True,i just dont like the start of the game where e...,NaN,2014-03-14,2014,-0.350000,0
7162,486259317,http://steamcommunity.com/id/486259317,,"Posted February 13, 2014.","Last edited February 13, 2014.",4000,No ratings yet,True,Think of one reason to not like this game.,NaN,2014-02-13,2014,-0.400000,0
10917,legendaneo,http://steamcommunity.com/id/legendaneo,,"Posted June 30, 2014.",,231430,1 of 3 people (33%) found this review helpful,True,Great Fast Past Strat game.8/10Needs more base...,NaN,2014-06-30,2014,-0.064286,0
10266,akopinoy,http://steamcommunity.com/id/akopinoy,,"Posted October 23, 2014.",Last edited May 22.,65980,No ratings yet,False,♥♥♥♥ this ♥♥♥♥. They shouldve made the dlc fre...,NaN,2014-10-23,2014,0.000000,1
3342,AngelicPanda,http://steamcommunity.com/id/AngelicPanda,,"Posted February 7, 2015.",,342380,1 of 1 people (100%) found this review helpful,True,The game is great with it's images and storyli...,NaN,2015-02-07,2015,0.115909,2


In [24]:
# Filtrado de campos a utilizar
df_reviews = df_reviews[['item_id', 'user_id', 'recommend', 'year_review', 'polarity', 'sentiment']]
df_reviews.sample(5)

,item_id,user_id,recommend,year_review,polarity,sentiment
3223,440,76561198068176506,True,2015,0.058333,2
791,238320,76561198055370293,True,2014,0.027971,2
5844,218620,76561198092448174,False,0,0.000000,1
24598,218620,TomWays,True,2015,0.000000,1
13001,355150,I_am_lion,True,0,0.262500,2


### Verificación de tipos de datos - Dataset REVIEWS

In [25]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   item_id      59305 non-null  object  
 1   user_id      59333 non-null  object  
 2   recommend    59305 non-null  object  
 3   year_review  59333 non-null  int64   
 4   polarity     59333 non-null  float64 
 5   sentiment    59333 non-null  category
dtypes: category(1), float64(1), int64(1), object(3)
memory usage: 2.8+ MB


In [26]:
# Conversión de tipos de datos
df_reviews['item_id'] = pd.to_numeric(df_reviews['item_id'], errors='coerce')
df_reviews['recommend'] = df_reviews['recommend'].astype(bool)
df_reviews['sentiment'] = pd.to_numeric(df_reviews['sentiment'], errors='coerce')
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      59305 non-null  float64
 1   user_id      59333 non-null  object 
 2   recommend    59333 non-null  bool   
 3   year_review  59333 non-null  int64  
 4   polarity     59333 non-null  float64
 5   sentiment    59333 non-null  int64  
dtypes: bool(1), float64(2), int64(2), object(1)
memory usage: 2.8+ MB


### Verificación de valores duplicados - Dataset REVIEWS

In [27]:
df_reviews.duplicated().sum()

874

In [28]:
# Eliminación de duplicados
df_reviews = df_reviews.drop_duplicates().reset_index(drop=True)
df_reviews.duplicated().sum()

0

### Verificación de valores nulos - Dataset REVIEWS

In [29]:
df_reviews.isnull().sum()

item_id        28
user_id         0
recommend       0
year_review     0
polarity        0
sentiment       0
dtype: int64

In [30]:
df_reviews = df_reviews.dropna(subset=['item_id'])
df_reviews.isnull().sum()

item_id        0
user_id        0
recommend      0
year_review    0
polarity       0
sentiment      0
dtype: int64

## Dataset USAGE

### Ingeniería de características - Dataset USAGE

In [31]:
df_usage = user_items
df_usage.sample(2)

,user_id,items_count,steam_id,user_url,items
6947,76561197996426598,124,76561197996426598,http://steamcommunity.com/profiles/76561197996...,"[{'item_id': '2100', 'item_name': 'Dark Messia..."
7888,TheMeatyMate,155,76561198054694784,http://steamcommunity.com/id/TheMeatyMate,"[{'item_id': '70', 'item_name': 'Half-Life', '..."


In [32]:
# Desagregación del campo "items"

df_usage = user_items.explode('items')

df_usage = df_usage.reset_index(drop=True)
def obtener_elemento(diccionario, clave_busqueda):
    if isinstance(diccionario, dict):
        return diccionario.get(clave_busqueda)
    else:
        return diccionario

# Desaagregaremos cada campo por separado para evitar tiempos excesivos de procesamiento
df_usage['item_id'] = df_usage['items'].apply(lambda x: obtener_elemento(x, 'item_id'))
df_usage['item_name'] = df_usage['items'].apply(lambda x: obtener_elemento(x, 'item_name'))
df_usage['playtime_forever'] = df_usage['items'].apply(lambda x: obtener_elemento(x, 'playtime_forever'))
df_usage['playtime_2weeks'] = df_usage['items'].apply(lambda x: obtener_elemento(x, 'playtime_2weeks'))

df_usage.sample(2)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
3056886,wildamn,110,76561198034682026,http://steamcommunity.com/id/wildamn,"{'item_id': '63380', 'item_name': 'Sniper Elit...",63380,Sniper Elite V2,0.0,0.0
4848032,76561198075229670,44,76561198075229670,http://steamcommunity.com/profiles/76561198075...,"{'item_id': '205790', 'item_name': 'Dota 2 Tes...",205790,Dota 2 Test,0.0,0.0


In [33]:
# Filtrar campos a utilizar
df_usage = df_usage[['item_id', 'user_id', 'playtime_forever']]
df_usage.sample(2)

,item_id,user_id,playtime_forever
3837906,10190,HAPPYCHEF,3539.0
2255442,550,spotto666,36.0


### Verificación de tipos de datos - Dataset USAGE


In [34]:
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   user_id           object 
 2   playtime_forever  float64
dtypes: float64(1), object(2)
memory usage: 118.3+ MB


In [35]:
# Conversión de tipos de datos
df_usage['item_id'] = pd.to_numeric(df_usage['item_id'], errors='coerce')
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           float64
 1   user_id           object 
 2   playtime_forever  float64
dtypes: float64(2), object(1)
memory usage: 118.3+ MB


### Verficación de valores duplicados - Dataset USAGE

In [36]:
df_usage.duplicated().sum()

59209

In [37]:
# Eliminación de duplicados
df_usage = df_usage.drop_duplicates().reset_index(drop=True)
df_usage.duplicated().sum()

0

### Verificación de valores nulos - Dataset USAGE

In [38]:
df_usage.isnull().sum()

item_id             16714
user_id                 0
playtime_forever    16714
dtype: int64

In [39]:
df_usage = df_usage.dropna(subset=['item_id'])
df_usage.isnull().sum()

item_id             0
user_id             0
playtime_forever    0
dtype: int64

# Exportación de datasets limpios

In [40]:
df_games.to_csv('df_games.csv', index=False)
df_reviews.to_csv('df_reviews.csv', index=False)
df_usage.to_csv('df_usage.csv', index=False)

# Armado y exportación de dataframes para API

## Samples

In [41]:
df_games.sample(2)

,item_id,app_name,genres,year,price,developer
66135,291610,Growing Pains,Indie,2014,4.99,Smudged Cat Games Ltd
32561,664850,8-bit Adventure Anthology: Volume I,Adventure,2017,7.99,"General Arcade,Abstraction Games"


In [42]:
df_reviews.sample(2)

,item_id,user_id,recommend,year_review,polarity,sentiment
5734,310370.0,76561198074373301,True,2015,0.3,2
36502,240.0,ThyGlyphDevice,True,2014,0.0,1


In [43]:
df_usage.sample(2)

,item_id,user_id,playtime_forever
4366524,22380.0,76561198049390920,1118.0
4308543,221100.0,76561198045633274,382.0


## Endpoint 1

def PlayTimeGenre(genre: str): Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [44]:
# Armado de dataframe para la función
df_e1 = pd.merge(df_games, df_usage, on="item_id", how="inner")
df_e1 = df_e1[['genres', 'year','playtime_forever']]

# Serie que contiene el índice de cada fila con el max playtime de cada género
df_e1_indmax = df_e1.groupby('genres')['playtime_forever'].idxmax() 

# Usar los índices para obtener los años correspondientes
df_e1 = df_e1.loc[df_e1_indmax, ['genres', 'year', 'playtime_forever']] 

# Mostrar los años con el máximo playtime_forever por género
df_e1 = df_e1[['genres', 'year']]
df_e1.to_csv('df_e1.csv', index=False)
df_e1

,genres,year
3160546,Action,2004
956218,Adventure,2014
921209,Animation &amp; Modeling,2013
1762950,Audio Production,2014
1599612,Casual,2014
2165742,Design &amp; Illustration,2012
1689391,Early Access,2014
947205,Education,2014
2133429,Free to Play,2013
27418,Indie,2006


In [45]:
# Probamos el dataframe para la función
df_func1 = df_e1[df_e1['genres'] == 'Action']
df_func1

,genres,year
3160546,Action,2004


In [46]:
# Definir la función
def PlayTimeGenre(input_genre: str):
    try:
        df_e1 = pd.read_csv("df_e1.csv")                # Lectura del df
        df_e1 = df_e1[df_e1["genres"] == input_genre]   # Filtrar por input
        
        output_year = df_e1.loc[df_e1['year'].idxmax(), 'year']
    
        return {f"Año de lanzamiento con más horas jugadas para {input_genre}": output_year}
    except Exception as e:
        return {"error": str(e)}

In [47]:
# Probamos la funcion
input_genre = "Strategy"
print(PlayTimeGenre(input_genre))

{'Año de lanzamiento con más horas jugadas para Strategy': 2010}


## Endpoint 2

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [48]:
df_e2 = pd.merge(df_games, df_usage, on="item_id", how="inner") # Unir datasets
df_e2 = df_e2[['genres', 'year', 'user_id','playtime_forever']] # Filtrar campos

# Obtener el índice de la fila con el máximo valor de playtime_forever para cada género
df_e2_indmax = df_e2.groupby('genres')['playtime_forever'].idxmax()
# Usar los índices para obtener los años correspondientes
# Utilzaremos el anio de lanzamiento para tomar el playtime
df_e2 = df_e2.loc[df_e2_indmax, ['genres', 'year', 'user_id', 'playtime_forever']]

In [49]:
df_e2_users = df_e2[['genres', 'user_id']]
df_e2_users.to_csv('df_e2_users.csv', index=False)
df_e2_users

,genres,user_id
3160546,Action,76561197977470391
956218,Adventure,DONTFUCKINGCLICKTHIS
921209,Animation &amp; Modeling,76561198059330972
1762950,Audio Production,Lickidactyl
1599612,Casual,76561198101480347
2165742,Design &amp; Illustration,76561198035718256
1689391,Early Access,76561198084846677
947205,Education,SeedyDog
2133429,Free to Play,Cow666
27418,Indie,wolop


In [50]:
df_e2_playtime = df_e2[['genres', 'year', 'user_id', 'playtime_forever']]
df_e2_playtime.to_csv('df_e2_playtime.csv', index=False)
df_e2_playtime

,genres,year,user_id,playtime_forever
3160546,Action,2004,76561197977470391,493791.0
956218,Adventure,2014,DONTFUCKINGCLICKTHIS,134223.0
921209,Animation &amp; Modeling,2013,76561198059330972,65427.0
1762950,Audio Production,2014,Lickidactyl,109916.0
1599612,Casual,2014,76561198101480347,74433.0
2165742,Design &amp; Illustration,2012,76561198035718256,102554.0
1689391,Early Access,2014,76561198084846677,1241.0
947205,Education,2014,SeedyDog,3082.0
2133429,Free to Play,2013,Cow666,32987.0
27418,Indie,2006,wolop,642773.0


## Endpoint 3

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [82]:
# Armado de dataframe para la fucnión
df_e3 = df_reviews[(df_reviews['recommend'] == True) & (df_reviews['sentiment'] >= 0)]
df_e3.loc[:, 'year_review'] = pd.to_numeric(df_e3['year_review'], errors='coerce')
df_3 = df_e3[df_e3['year_review'] != 0]
df_e3 = df_e3.groupby(['year_review', 'item_id'])['recommend'].count().reset_index()
df_e3.rename(columns={'recommend': 'recommend_count'}, inplace=True)
df_e3 = pd.merge(df_e3, df_games, on="item_id", how="left").sort_values(by='recommend_count', ascending=False)
df_e3 = df_e3.dropna(subset=['app_name'])
df_e3 = df_e3[['year_review','app_name', 'recommend_count']]

df_e3.to_csv('df_e3.csv', index=False)
df_e3

,year_review,app_name
4320,2015,Counter-Strike: Global Offensive
2853,2014,Counter-Strike: Global Offensive
2887,2014,Garry's Mod
17,0,Counter-Strike: Global Offensive
3822,2014,Rust
...,...,...
2434,2013,Post Apocalyptic Mayhem
2436,2013,SpaceChem
2437,2013,Dinner Date
2438,2013,Jamestown


In [52]:
# Probamos el dataframe para la funcion
df_func3 = df_e3[df_e3['year_review'] == 2015].head(3)
df_func3

,year_review,app_name,recommend_count
4320,2015,Counter-Strike: Global Offensive,1527
4353,2015,Garry's Mod,357
5265,2015,Grand Theft Auto V,215


In [53]:
# Definir la función
def UsersRecommend(input_year: int):
    try:
        df_e3 = pd.read_csv("df_e3.csv")
        output_top3 = df_e3[df_e3['year_review'] == input_year].head(3)
        output_top3_list = [{"Puesto {}: {}".format(i+1, game)} for i, game in enumerate(output_top3['app_name'])]
        return output_top3_list
    except Exception as e:
        return {"error": str(e)}

In [54]:
# Probamos la funcion
input_year = 2015
print(UsersRecommend(input_year))

[{'Puesto 1: Counter-Strike: Global Offensive'}, {"Puesto 2: Garry's Mod"}, {'Puesto 3: Grand Theft Auto V'}]


## Endpoint 4

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [55]:
# Armado de dataframe para la fucnión
df_e4 = df_e3 # Obswrve que es el mismo dataframe que para la función anterior
df_e4.to_csv('df_e4.csv', index=False)
df_e4

,year_review,app_name,recommend_count
4320,2015,Counter-Strike: Global Offensive,1527
2853,2014,Counter-Strike: Global Offensive,1068
2887,2014,Garry's Mod,757
17,0,Counter-Strike: Global Offensive,638
3822,2014,Rust,389
...,...,...,...
2434,2013,Post Apocalyptic Mayhem,1
2436,2013,SpaceChem,1
2437,2013,Dinner Date,1
2438,2013,Jamestown,1


In [56]:
# Probamos el dataframe para la funcion
df_func4 = df_e3[df_e3['year_review'] == 2015].tail(3)
df_func4

,year_review,app_name,recommend_count
4908,2015,Fallen Enchantress: Legendary Heroes,1
5358,2015,Colin McRae Rally,1
6133,2015,The Quest,1


In [57]:
# Definir la función
def UsersNotRecommend(input_year: int):
    try:
        df_e4 = pd.read_csv("df_e4.csv")
        output_last3 = df_e4[df_e4['year_review'] == input_year].tail(3)
        output_last3_list = [{"Puesto {}: {}".format(i+1, game)} for i, game in enumerate(output_last3['app_name'])]
        return output_last3_list
    except Exception as e:
        return {"error": str(e)}

In [58]:
# Probamos la funcion
input_year = 2015
print(UsersNotRecommend(input_year))

[{'Puesto 1: Fallen Enchantress: Legendary Heroes'}, {'Puesto 2: Colin McRae Rally'}, {'Puesto 3: The Quest'}]


## Endpoint 5

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [59]:
# Armado de dataframe para la función
df_e5 = df_reviews[df_reviews['year_review'] != 0].copy()
# Crear columnas 'negative', 'neutral' y 'positive'
df_e5['negative'] = (df_e5['sentiment'] == 1).astype(int)
df_e5['neutral'] = (df_e5['sentiment'] == 0).astype(int)
df_e5['positive'] = (df_e5['sentiment'] == 2).astype(int)

# Dado que no se solicitan resultados por item_id, tomaremos como anio de lanzamiento el year_review ya que lo que se solicita es la suma de los sentiments
# Agrupar y pivotar para contar los valores según year_review
df_e5 = df_e5.groupby('year_review')[['negative', 'neutral', 'positive']].sum()
df_e5 = df_e5.rename_axis('year').reset_index()

df_e5.to_csv('df_e5.csv', index=False)
df_e5

,year,negative,neutral,positive
0,2010,8,11,47
1,2011,70,94,366
2,2012,215,206,780
3,2013,1362,1230,4121
4,2014,4736,4389,12709
5,2015,4257,4149,9748


In [60]:
# Probamos el dataframe para la funcion
year = 2015
df_func5 = df_e5[df_e5['year'] == year]
df_func5

,year,negative,neutral,positive
5,2015,4257,4149,9748


In [61]:
# Definir la función
def SentimentAnalysis(input_year: int):
    try:
        df_e5 = pd.read_csv("df_e5.csv")

        value_negative = df_e5['negative'].values[0]
        value_neutral = df_e5['neutral'].values[0]
        value_positive = df_e5['positive'].values[0]

        output_sentiment_list = f"Para el año {input_year} se registran los siguientes valores: negative: {value_negative}, neutral: {value_neutral}, neutral: {value_positive}"       
        return output_sentiment_list
    except Exception as e:
        return {"error": str(e)}

In [62]:
# Probamos la funcion
input_year = 2015
print(SentimentAnalysis(input_year))

Para el año 2015 se registran los siguientes valores: negative: 8, neutral: 11, neutral: 47
